In [1]:
using QuadraticTeamDecisionSolver, Distributions, Statistics, LinearAlgebra, Plots, LaTeXStrings

In [2]:
#replace with desired directory
saveTo = "/home/fe/michelena/Schreibtisch/thesis/progress"

"/home/fe/michelena/Schreibtisch/thesis/progress"

In [3]:
N = 2;
σᵣ = 1.5;
p = QuadTeamProblem(N, N * N * ones(Int64, N), ones(Int64, N), ComplexF64);
t = teamMMSEproblem(
	σᵣ,
	zeros(Float64, N),
	ones(Float64, N),
	zeros(Float64, N),
	0.5 * ones(Float64, N),
	0.0 * ones(Float64, N),
);


In [4]:
function solver!(
	p,
	w,
	Y,
	R,
	r;
	iterations = 10,
	h = 0.05,
	λ = 0.01,
)
	k(x, y; h = h) = exp((-norm(x - y)^2) / h)
	regression(Y, X) = kernelInterpolation(k, Y, X, λ = λ)
	regressor(w, X, x) = kernelFunction(k, w, X, x)
	return empiricalJacobiSolver!(
		p,
		w,
		Y,
		R,
		r,
		regression,
		regressor,
		iterations = iterations,
	)
end


solver! (generic function with 1 method)

In [5]:
m = 1000;
iterations = 10;
Y, R, r = generateTeamMMSEsamples(p, t, m);

In [6]:
w_jacobi = [[rand(ComplexF64, m)] for _ in 1:p.N];
solver!(p, w_jacobi, Y, R, r);

InterruptException: InterruptException:

In [7]:
conv_jacobi = [norm.(diff(w_jacobi[i])) for i in 1:p.N];

In [8]:
testR = reformatR(p.N, m, R);
testr = reformatr(p.N, m, r);
testW_jacobi = reformatU(p.N, m, iterations, w_jacobi);

BoundsError: BoundsError: attempt to access 7-element Vector{Vector{ComplexF64}} at index [8]

In [9]:
perf_jacobi = [urisk(testW_jacobi[k], testR, testr) for k in 1:iterations];

UndefVarError: UndefVarError: `testW_jacobi` not defined

In [10]:
conv = plot(
	conv_jacobi,
	xscale = :log10,
	yscale = :log10,
	xlabel = L"(k)",
	label = [L"||\mathbf{U}_{(k)}^1 - \mathbf{U}_{(k-1)}^1||" L"||\mathbf{U}_{(k)}^2 - \mathbf{U}_{(k-1)}^2||"],
);
perf = plot(
	perf_jacobi,
	yscale = :log10,
	xscale = :log10,
	xlabel = L"(k)",
	label = L"J^{(m)}(\mathbf{X}; (\mathbf{U}^1, \mathbf{U}^2)))",
);
plot(conv, perf, layout = (1, 2), size = (1000, 300))
savefig(saveTo * "/convergence.png");
display(Plots.current())


UndefVarError: UndefVarError: `perf_jacobi` not defined

In [11]:
# Step 2: Create a 1x5 grid of subplots
plot_layout = plot(layout=(2, iterations), legend=false)

# Step 3: Generate histograms and add them to the grid
for k in 1:iterations
    h1 = histogram2d(
        w_jacobi[1][k],
        normalize=:pdf,
        show_empty_bins=false,
        normed=true,
        aspect_ratio=1,
    )
    histogram2d!(
        plot_layout[k],
        w_jacobi[1][k],
        normalize=:pdf,
        show_empty_bins=false,
        normed=true,
        aspect_ratio=1,
    )

    h2 = histogram2d(
        w_jacobi[2][k],
        normalize=:pdf,
        show_empty_bins=false,
        normed=true,
        aspect_ratio=1,
    )
    histogram2d!(
        plot_layout[2, k],
        w_jacobi[2][k],
        normalize=:pdf,
        show_empty_bins=false,
        normed=true,
        aspect_ratio=1,
    )
end

# Step 4: Show the plot
plot(plot_layout, size=(2500, 500))
savefig(saveTo * "/evolution.png");
display(Plots.current())

BoundsError: BoundsError: attempt to access 7-element Vector{Vector{ComplexF64}} at index [8]

In [12]:
sinrRange = 10 .^ range(start = -10, stop = 10, length = 10);
sinrResult = [
	sinrExperiment(
		p,
		solver!,
		sinr;
		samples = 1000,
		iterations = 10,
		bandwidth = 0.5,
		regularizer = 0.2,
		eps = 0.0,
	) for sinr in sinrRange
];


In [ ]:
sinrplt = plot(size = (700, 500))
for (sinr, curve) in zip(sinrRange, sinrResult)
	plot!(sinrplt, curve, yscale = :log10, xscale = :log10, legend = false)

	# Get the last point of the current curve to place the label
	x_label = iterations-2
	y_label = last(curve)

	# Add the label on top of the curve
	annotate!(
		sinrplt,
		x_label,
		y_label,
		text(latexstring("\\sigma = $(sinr)"), 8, :left, :white, :black, 8),
	)
end
savefig(saveTo * "/sinrplt.png");
display(Plots.current())


In [ ]:
hRange = 10 .^ range(start = -10, stop = 10, length = 10);
hResult = [
	bandwidthExperiment(
		p,
		solver!,
		h;
		samples = 1000,
		iterations = 10,
		sinr = 1.5,
		regularizer = 0.2,
		eps = 0.0,
	) for h in hRange
];

In [ ]:
hplt = plot(size = (700, 500))
for (h, curve) in zip(sinrRange, hResult)
	plot!(hplt, curve, yscale = :log10, xscale = :log10, legend = false)

	# Get the last point of the current curve to place the label
	x_label = iterations-2
	y_label = last(curve)

	# Add the label on top of the curve
	annotate!(
		sinrplt,
		x_label,
		y_label,
		text(latexstring("h = $(h)"), 8, :left, :white, :black, 8),
	)
end
savefig(saveTo * "/hplt.png");
display(Plots.current())